#  Neural Networks: Regression on House Pricing Dataset
We consider a reduced version of a dataset containing house sale prices for King County, which includes Seattle. It includes homes sold between May 2014 and May 2015.

https://www.kaggle.com/harlfoxem/housesalesprediction

For each house we know 18 house features (e.g., number of bedrooms, number of bathrooms, etc.) plus its price, that is what we would like to predict.

In [ ]:
#put here your ``numero di matricola''
numero_di_matricola = 2019285

## Insert your ID number ("numero di matricola") below

In [1]:
#put here your ``numero di matricola''
numero_di_matricola = 2019285

In [2]:
#import all packages needed
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Load the data, remove data samples/points with missing values (NaN) and take a look at them.

In [3]:
#load the data
df = pd.read_csv('kc_house_data.csv', sep = ',')

#remove the data samples with missing values (NaN)
df = df.dropna() 

df.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,3.164000e+03,3.164000e+03,3164.000000,3164.000000,3164.000000,3.164000e+03,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000
mean,4.645240e+09,5.354358e+05,3.381163,2.071903,2070.027813,1.525054e+04,1.434893,0.009798,0.244311,3.459229,7.615676,1761.252212,308.775601,1967.489254,94.668774,98077.125158,47.557868,-122.212337,1982.544564,13176.302465
std,2.854203e+09,3.809004e+05,0.895472,0.768212,920.251879,4.254457e+04,0.507792,0.098513,0.776298,0.682592,1.166324,815.934864,458.977904,28.095275,424.439427,54.172937,0.140789,0.139577,686.256670,25413.180755
min,1.000102e+06,7.500000e+04,0.000000,0.000000,380.000000,6.490000e+02,1.000000,0.000000,0.000000,1.000000,3.000000,380.000000,0.000000,1900.000000,0.000000,98001.000000,47.177500,-122.514000,620.000000,660.000000
25%,2.199775e+09,3.150000e+05,3.000000,1.500000,1430.000000,5.453750e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1190.000000,0.000000,1950.000000,0.000000,98032.000000,47.459575,-122.324250,1480.000000,5429.500000
50%,4.027701e+09,4.450000e+05,3.000000,2.000000,1910.000000,8.000000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1545.000000,0.000000,1969.000000,0.000000,98059.000000,47.572500,-122.226000,1830.000000,7873.000000
75%,7.358175e+09,6.402500e+05,4.000000,2.500000,2500.000000,1.122250e+04,2.000000,0.000000,0.000000,4.000000,8.000000,2150.000000,600.000000,1990.000000,0.000000,98117.000000,47.680250,-122.124000,2360.000000,10408.250000
max,9.839301e+09,5.350000e+06,8.000000,6.000000,8010.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,12.000000,6720.000000,2620.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,5790.000000,425581.000000


Extract input and output data. We want to predict the price by using features other than id as input.

In [5]:
Data = df.values
# m = number of input samples
m = Data.shape[0]
print("Amount of data:",m)
Y = Data[:m,2]
X = Data[:m,3:]

Amount of data: 3164


## Data Pre-Processing

We split the data into 3 parts: one will be used for training and choosing the parameters, one for choosing among different models, and one for testing. The part for training and choosing the parameters will consist of $m_{train}=200$ samples, the one for choosing among different models will consist of $m_{val}=100$ sampels, while the other part consists of $m_{test}=m - m_{traing} - m_{val}$ samples ($m$ is the total number of samples in the data).

In [7]:
# Split data into train (2/3 of samples), validation (1/6 of samples), and test data (the rest)
m_train = int(2./3.*m)
m_val = int((m-m_train)/2.)
m_test = m - m_train - m_val
print("Amount of data for training and deciding parameters:",m_train)
print("Amount of data for validation (choosing among different models):",m_val)
print("Amount of data for test:",m_test)
from sklearn.model_selection import train_test_split

Xtrain_and_val, Xtest, Ytrain_and_val, Ytest = train_test_split(X, Y, test_size=m_test/m, random_state=numero_di_matricola)
Xtrain, Xval, Ytrain, Yval = train_test_split(Xtrain_and_val, Ytrain_and_val, test_size=m_val/(m_train+m_val), random_state=numero_di_matricola)

Amount of data for training and deciding parameters: 2109
Amount of data for validation (choosing among different models): 527
Amount of data for test: 528


Let's standardize the data.

In [8]:
# Data pre-processing
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(Xtrain)
Xtrain_scaled = scaler.transform(Xtrain)
Xtrain_and_val_scaled = scaler.transform(Xtrain_and_val)
Xval_scaled = scaler.transform(Xval)
Xtest_scaled = scaler.transform(Xtest)

## Neural Networks
Let's start by learning a simple neural network with 1 hidden node.
Note: we are going to use the input parameter solver='lbfgs' and random_state=numero_di_matricola to fix the random seed (so results are reproducible).

In [20]:
#let's load the MLPRegressor

from sklearn.neural_network import MLPRegressor

#let's define the model
mlp = MLPRegressor( hidden_layer_sizes=(1,), solver = 'lbfgs', random_state = numero_di_matricola)

#let's learn the model on training data
mlp.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print('1- coefficient of determination on training data: '+str(1-mlp.score(Xtrain_scaled,Ytrain)))

#let's print the error (1 - R^2) on validation data
print('1- coefficient of determination on validation data: '+str(1-mlp.score(Xval_scaled,Yval)))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(mlp.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
print(mlp.intercepts_)

1- coefficient of determination on training data: 0.26432589860935896
1- coefficient of determination on validation data: 0.3993956017038188
[array([[-286.42604869],
       [ 275.80854901],
       [ 701.47754136],
       [  60.13597384],
       [ -36.00603178],
       [ 731.42871035],
       [ 347.58511158],
       [ 116.99813774],
       [ 893.29570813],
       [ 677.32413976],
       [ 204.98308093],
       [-627.07930077],
       [  42.43026444],
       [-240.31810162],
       [ 636.78256752],
       [-257.32492375],
       [ 143.33951491],
       [ -90.93810337]]), array([[126.72194113]])]
[array([4269.16669622]), array([3288.26615873])]


## Neural Networks vs Linear Models

Let's learn a linear model on the other same data and compare the results with the simple NN above.

In [29]:
from sklearn import linear_model

LR = linear_model.LinearRegression()

LR.fit(Xtrain_scaled,Ytrain)

print('1- coefficient of determination on training data: '+str(1-LR.score(Xtrain_scaled,Ytrain)))
print('1- coefficient of determination on validation data: '+str(1-LR.score(Xval_scaled,Yval)))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(LR.coef_)

#let's print the coefficient for the bias (i.e., the bias)
print(LR.intercept_)


1- coefficient of determination on training data: 0.2657492430694852
1- coefficient of determination on validation data: 0.39877003142210965
[-38677.49477152  32713.36460133  90409.35001079   6859.08560932
  -4085.99882982  92587.73895424  44440.00957999  14911.73685578
 107790.57245699  86825.86271749  27106.62806347 -76873.32418213
   6423.81614015 -29674.87850048  79703.0485821  -30130.4136688
  18899.74928706 -11342.04115164]
546036.8809862598


Is there a way to make a NN network learn a linear model?

Let's first check what the loss used by MLPRegressor...

In [26]:
#let's write the code to learn a linear model with NN: how? maybe let's use an activation function that is identity

#let's define the model
mlp_lr = MLPRegressor(activation = 'identity',hidden_layer_sizes=(1,), solver = 'lbfgs', random_state = numero_di_matricola)

#let's learn the model on training data
mlp_lr.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print('1- coefficient of determination on training data: '+str(1-mlp_lr.score(Xtrain_scaled,Ytrain)))

#let's print the error (1 - R^2) on validation data
print('1- coefficient of determination on validation data: '+str(1-mlp_lr.score(Xval_scaled,Yval)))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(mlp_lr.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
print(mlp_lr.intercepts_)

1- coefficient of determination on training data: 0.26574937651177133
1- coefficient of determination on validation data: 0.3987066710654992
[array([[  476.03034827],
       [ -403.28215265],
       [-1112.71732546],
       [  -83.93970163],
       [   51.71000684],
       [-1137.4314383 ],
       [ -546.83388317],
       [ -183.43535436],
       [-1323.94682021],
       [-1068.85878531],
       [ -332.34544972],
       [  945.28573289],
       [  -78.83109383],
       [  364.73768431],
       [ -980.10934728],
       [  370.02570093],
       [ -232.05795824],
       [  139.56910416]]), array([[-81.32855266]])]
[array([-6633.54347817]), array([6555.60353918])]


Note that there is an $\ell_2$ regularization term in MLPRegressor. What about making it smaller?

In [30]:
#let's define the model
mlp_lr = MLPRegressor(activation = 'identity',hidden_layer_sizes=(1,), solver = 'lbfgs',alpha = 0.0000001, random_state = numero_di_matricola)

#let's learn the model on training data
mlp_lr.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print('1- coefficient of determination on training data: '+str(1-mlp_lr.score(Xtrain_scaled,Ytrain)))

#let's print the error (1 - R^2) on validation data
print('1- coefficient of determination on validation data: '+str(1-mlp_lr.score(Xval_scaled,Yval)))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(mlp_lr.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
print(mlp_lr.intercepts_)

1- coefficient of determination on training data: 0.26574932770349147
1- coefficient of determination on validation data: 0.39878644075688585
[array([[  475.53891729],
       [ -401.75878358],
       [-1111.47878601],
       [  -83.93348631],
       [   50.79097056],
       [-1138.41367685],
       [ -545.49273256],
       [ -183.25610546],
       [-1324.2342378 ],
       [-1067.40409752],
       [ -332.44636879],
       [  944.38759   ],
       [  -79.35400732],
       [  364.69765385],
       [ -979.71494634],
       [  370.63076879],
       [ -233.84122177],
       [  139.02203503]]), array([[-81.36014946]])]
[array([-6630.68951416]), array([6553.56472908])]


## More Complex NNs

Let's try more complex NN, for example increasing the number of nodes in the only hidden layer, or increasing the number of hidden layers.

Let's build a NN with 2 nodes in the only hidden layer

In [31]:
#let's build a NN with 2 nodes in the only hidden layer


#let's define the model
mlp_2n = MLPRegressor( hidden_layer_sizes=(2,), solver = 'lbfgs', random_state = numero_di_matricola)

#let's learn the model on training data
mlp_2n.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print('1- coefficient of determination on training data: '+str(1-mlp_2n.score(Xtrain_scaled,Ytrain)))

#let's print the error (1 - R^2) on validation data
print('1- coefficient of determination on validation data: '+str(1-mlp_2n.score(Xval_scaled,Yval)))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(mlp_2n.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
print(mlp_2n.intercepts_)

1- coefficient of determination on training data: 0.2147850535235294
1- coefficient of determination on validation data: 0.31837956954662006
[array([[-137.91175118,  110.83290975],
       [ 134.12205098,  -45.9634789 ],
       [ 364.86594039, -243.19767558],
       [   0.60953176,   28.85919544],
       [ -82.20690691,  124.30151162],
       [ 228.94074765, -154.52900943],
       [ 170.78171145, -128.16896333],
       [  38.98868394,   21.15364485],
       [ 409.95658103,  -88.50669909],
       [ 373.37104778, -260.30740463],
       [  67.98670564,  -25.51178652],
       [-398.09980631,  228.45286509],
       [  13.45347583,  -10.14627134],
       [-207.47988007,  134.56422356],
       [ 290.26521542,  -21.45329987],
       [-211.50064758,  104.43492468],
       [  70.31482766,   19.50825948],
       [-166.82859052,  135.78006802]]), array([[368.8099617 ],
       [288.79829668]])]
[array([934.09534165, 509.43594914]), array([336.51560696])]


Let's build a NN with 5 nodes in the only hidden layer

In [32]:
#let's build a NN with 5 nodes in the only hidden layer

#let's define the model
mlp_5n = MLPRegressor( hidden_layer_sizes=(5,), solver = 'lbfgs', random_state = numero_di_matricola)

#let's learn the model on training data
mlp_5n.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print('1- coefficient of determination on training data: '+str(1-mlp_5n.score(Xtrain_scaled,Ytrain)))

#let's print the error (1 - R^2) on validation data
print('1- coefficient of determination on validation data: '+str(1-mlp_5n.score(Xval_scaled,Yval)))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(mlp_5n.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
print(mlp_5n.intercepts_)

1- coefficient of determination on training data: 0.1770385138884648
1- coefficient of determination on validation data: 0.2990425788191562
[array([[ -65.06548978,   81.68164082,   -2.32284965,  100.42718139,
         -70.61669363],
       [  12.98638477,   -0.90753773,   60.95069611,   21.01431417,
          46.86634846],
       [   8.86450006,  -43.89658038,  307.84462298, -215.97085646,
         170.01507702],
       [ -57.87635467,    8.44535974,  135.51667505,  -62.55688594,
         -93.73503236],
       [ -72.06504387, -462.26701442, -144.95230465,  301.01605012,
         -33.03173038],
       [ -90.48628711, -190.18575721,  134.11031835, -174.07328618,
         137.07893402],
       [ -76.56747597,  -68.47909217,  105.56037297, -104.90909581,
         115.43740104],
       [-185.35539184,  -70.63753433,   66.59782832,  221.7437651 ,
         -34.95172465],
       [ 101.72196135,  295.25450055,  204.19806894,  -71.91735758,
         220.00622206],
       [  -8.89796927,   33.594

/Users/nicolafarronato/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:471: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Note that with a smaller number of iterations we had a larger error on training set but a smaller error on validation data -> "early stopping is a form of regularization"

Let's build a NN with 10 nodes in the only hidden layer

In [35]:
#let's build a NN with 10 nodes in the only hidden layer

#let's define the model
#mlp_10n = MLPRegressor( hidden_layer_sizes=(10,), solver = 'lbfgs', random_state = numero_di_matricola)
#increase iterations
mlp_10n = MLPRegressor( hidden_layer_sizes=(10,), solver = 'lbfgs', random_state = numero_di_matricola, max_iter = 5000)


#let's learn the model on training data
mlp_10n.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print('1- coefficient of determination on training data: '+str(1-mlp_10n.score(Xtrain_scaled,Ytrain)))

#let's print the error (1 - R^2) on validation data
print('1- coefficient of determination on validation data: '+str(1-mlp_10n.score(Xval_scaled,Yval)))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(mlp_10n.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
print(mlp_10n.intercepts_)

1- coefficient of determination on training data: 0.07115337159853485
1- coefficient of determination on validation data: 0.21675275013286577
[array([[   22.1138908 ,   -46.98341748,   -32.6271824 ,    36.04774482,
         -112.58133157,   -57.04538082,    61.22222601,    34.83384073,
            8.61226358,    41.73672045],
       [ -236.90689182,    12.34447579,  -149.45764589,  -231.38996925,
           65.43075506,   173.58704216,   -46.88997744,   -80.6327762 ,
          -86.93823786,  -169.37074656],
       [   62.57843999,   129.62647606,   100.65897129,    14.57336832,
          -29.83890614,    40.99590198,   138.08101438,    59.1290489 ,
          189.83356303,   306.04234483],
       [ -191.12424088,     3.87564858,  -180.21350759,  -197.61192895,
          -27.86009812,   238.05980767,   232.25058443,   229.4089445 ,
          120.80206977,    78.72235681],
       [ -114.68225853,  -262.29618171,   -41.91147373,  -138.52619419,
          603.73375094,    97.97449993,  -230

Let's build a NN with 100 nodes in the only hidden layer. Note that this is the default!

In [37]:
#let's build a NN with 100 nodes in the only hidden layer

#let's define the model
#mlp_10n = MLPRegressor( hidden_layer_sizes=(10,), solver = 'lbfgs', random_state = numero_di_matricola)
#increase iterations
mlp_100n = MLPRegressor( hidden_layer_sizes=(100,), solver = 'lbfgs', random_state = numero_di_matricola, max_iter = 1000)


#let's learn the model on training data
mlp_100n.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print('1- coefficient of determination on training data: '+str(1-mlp_100n.score(Xtrain_scaled,Ytrain)))

#let's print the error (1 - R^2) on validation data
print('1- coefficient of determination on validation data: '+str(1-mlp_100n.score(Xval_scaled,Yval)))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(mlp_100n.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
print(mlp_100n.intercepts_)

1- coefficient of determination on training data: 0.008455614703022807
1- coefficient of determination on validation data: 0.4117290205532711
[array([[ 162.88523338,   69.70698043,  -43.23680023, ...,  -62.2663714 ,
          11.73177312,  -48.16315467],
       [ -97.17445877,  -42.04394087, -130.55099555, ...,   24.85456256,
        -232.22435201,  176.56871782],
       [-120.93891427,  -97.40845345,   65.77730149, ..., -205.79903001,
          61.72230029,   21.90943647],
       ...,
       [ 103.80718352,   39.36431581,   77.36303639, ..., -175.13434637,
          88.54886048, -259.58547436],
       [  40.54158897,  149.25307175,   62.72664511, ...,  145.65571347,
         -82.95689123,  148.55308654],
       [ -90.6529922 ,   30.94280756,   97.38576295, ...,   61.76679245,
        -115.72633501, -175.30975992]]), array([[ 283.21879721],
       [-669.66349951],
       [-333.52346908],
       [ 364.27317791],
       [ 312.8570524 ],
       [ 317.08552998],
       [-360.37203561],
   

/Users/nicolafarronato/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:471: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Let's try 2 layers, 1 node each

In [40]:
#let's build a NN with 2 hidden layers, 1 node each

#let's define the model
#mlp_10n = MLPRegressor( hidden_layer_sizes=(10,), solver = 'lbfgs', random_state = numero_di_matricola)
#increase iterations
mlp_2l1n = MLPRegressor( hidden_layer_sizes=(1,2), solver = 'lbfgs', random_state = numero_di_matricola, max_iter = 2000)


#let's learn the model on training data
mlp_2l1n.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print('1- coefficient of determination on training data: '+str(1-mlp_2l1n.score(Xtrain_scaled,Ytrain)))

#let's print the error (1 - R^2) on validation data
print('1- coefficient of determination on validation data: '+str(1-mlp_2l1n.score(Xval_scaled,Yval)))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(mlp_2l1n.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
print(mlp_2l1n.intercepts_)

1- coefficient of determination on training data: 0.2509291947067428
1- coefficient of determination on validation data: 0.39846071138505756
[array([[-1107.17919078],
       [ 3033.52415987],
       [ 5584.33691351],
       [-2725.32963372],
       [ 1401.48663661],
       [ 5517.2059507 ],
       [ 2023.37330424],
       [ 3951.78481295],
       [ 7699.50611665],
       [ 5715.16305874],
       [ 1048.09919605],
       [-4215.92157656],
       [ 1138.25612926],
       [-1745.69695201],
       [ 7410.73878623],
       [-3991.56396449],
       [ 4110.48382393],
       [-3246.84410016]]), array([[-0.74239411,  0.03828127]]), array([[ -1.36707828],
       [469.27798594]])]
[array([1888.9227425]), array([ -0.98815288, 746.75075159]), array([294.56944642])]


Let's try 2 layers, 2 nodes each

In [ ]:
#let's build a NN with 2 layers, 2 nodes each

#COMPLETE

Let's try 2 layers, 10 nodes each

In [ ]:
#let's build a NN with 2 layers, 10 nodes each

#COMPLETE

Let's try 2 layers, 100 nodes each

In [ ]:
#let's build a NN with 2 layers, 100 nodes each

#COMPLETE

So it seems that 1 layer (and default number of iterations) works best for this dataset. Let's try 5-fold cross-validation with number of nodes in the hidden layer between 1 and 20.
Note that we use train and validation data together, since we are doing cross-validation.

In [43]:
from sklearn.model_selection import GridSearchCV

mlp_cv = MLPRegressor()
param_grid = {'hidden_layer_sizes': [i for i in range (1,21)],
             'activation': ['relu'],
             'solver': ['lbfgs'],
             'random_state' : [numero_di_matricola]}
mlp_GS = GridSearchCV(mlp_cv, param_grid = param_grid, cv=5, verbose=True)
mlp_GS.fit(Xtrain_and_val_scaled,Ytrain_and_val)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/nicolafarronato/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:471: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Users/nicolafarronato/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:471: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Users/nicolafarronato/opt/anaconda3/lib/python3.8/

GridSearchCV(cv=5, estimator=MLPRegressor(),
             param_grid={'activation': ['relu'],
                         'hidden_layer_sizes': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10,
                                                11, 12, 13, 14, 15, 16, 17, 18,
                                                19, 20],
                         'random_state': [2019285], 'solver': ['lbfgs']},
             verbose=True)

Now let's check what is the best parameter, and compare the best NNs with the linear model (learned on train and validation) on test data.

In [45]:
#let's print the best model according to grid search
print(mlp_GS.best_estimator_)

#let's print the error 1-R^2 for the best model
print(mlp_GS.best_score_)

MLPRegressor(hidden_layer_sizes=16, random_state=2019285, solver='lbfgs')
0.8207814374734955


Let compare the error of the best NN on train and validation and on test data.

In [49]:
print(mlp_GS.best_estimator_.score(Xtrain_and_val_scaled,Ytrain_and_val))
print(mlp_GS.best_estimator_.score(Xtest_scaled,Ytest))

0.9143199275692684
0.7983869383717074


Now let's learn the linear model on train and validation, and get error (1-R^2) on train and validation and on test data.

In [ ]:
#COMPLETE

Note: MLPRegressor has several other parameters!